# Deep Learning Backware Propagation

## Import Packages

In [1]:
import torch

/home/babi/miniconda3/envs/tenx_week3/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Creating Layers

In [2]:
class DenseLayer:
    """
    A simple dense layer for neural networks.

    Parameters:
    - n_inputs (int): Number of input features.
    - n_neurons (int): Number of neurons in the layer.

    Attributes:
    - weights (torch.Tensor): The weights matrix for the layer.
    - biases (torch.Tensor): The biases vector for the layer.
    - output (torch.Tensor): The output values after a forward pass.

    Methods:
    - forward(inputs): Perform a forward pass through the layer.
    """

    def __init__(self, n_inputs, n_neurons):
        """
        Initialize a DenseLayer instance.

        Parameters:
        - n_inputs (int): Number of input features.
        - n_neurons (int): Number of neurons in the layer.
        """
        # Initialize weights and biases
        self.weights = 0.01 * torch.rand(n_inputs, n_neurons)
        self.biases = torch.zeros((1, n_neurons))

    def forward(self, inputs):
        """
        Perform a forward pass through the layer.

        Parameters:
        - inputs (torch.Tensor): Input tensor for the forward pass.

        Returns:
        - output (torch.Tensor): Output tensor after the forward pass.
        """
        # Calculate output values from inputs, weights, and biases
        self.output = torch.matmul(inputs, self.weights) + self.biases
        return self.output


## Activation Functions

### ReLU

In [3]:
class Activation_ReLU:
    """
    Rectified Linear Unit (ReLU) activation function.

    The ReLU activation function is commonly used in neural networks to introduce
    non-linearity to the model.

    Methods:
    - forward(inputs): Perform a forward pass through the ReLU activation.
    """

    def forward(self, inputs):
        """
        Perform a forward pass through the ReLU activation.

        Parameters:
        - inputs (torch.Tensor): Input tensor for the forward pass.

        Returns:
        - output (torch.Tensor): Output tensor after the ReLU activation.
        """
        self.output = torch.max(torch.tensor(0), inputs)
        return self.output


### Sigmoid

In [4]:
class Activation_Sigmoid:
    """
    Sigmoid activation function.

    The sigmoid activation function is commonly used in neural networks to squash
    the output of a neuron between 0 and 1.

    Methods:
    - forward(inputs): Perform a forward pass through the sigmoid activation.
    """

    def forward(self, inputs):
        """
        Perform a forward pass through the sigmoid activation.

        Parameters:
        - inputs (torch.Tensor): Input tensor for the forward pass.

        Returns:
        - output (torch.Tensor): Output tensor after the sigmoid activation.
        """
        self.output = 1 / (1 + torch.exp(inputs * -1))
        return self.output


### Softmax

## Loss

In [5]:
class Loss_MeanSquareError:
    """
    Mean Square Error (MSE) loss function.

    The Mean Square Error loss is commonly used in regression problems to measure
    the average squared difference between predicted and true values.

    Methods:
    - forward(y_pred, y_true): Calculate the Mean Square Error.

    ```

    """

    def forward(self, y_pred, y_true):
        """
        Calculate the Mean Square Error (MSE).

        Parameters:
        - y_pred (torch.Tensor): Predicted values.
        - y_true (torch.Tensor): True values.

        Returns:
        - mse_loss (torch.Tensor): Mean Square Error loss.
        """
        mse_loss = torch.mean((y_pred - y_true)**2)
        return mse_loss

## Accuracy

In [6]:
class Accuracy:
    """
    Accuracy metric for classification problems.

    The Accuracy metric measures the proportion of correctly classified samples.

    Methods:
    - calculate(y_pred, y_true): Calculate the accuracy.
    """

    def calculate(self, y_pred, y_true):
        """
        Calculate the accuracy.

        Parameters:
        - y_pred (torch.Tensor): Predicted values.
        - y_true (torch.Tensor): True values.

        Returns:
        - accuracy (torch.Tensor): Accuracy value.
        """
        predictions = torch.argmax(y_pred, axis=1)

        if len(y_true.shape) == 2:
            y_true = torch.argmax(y_true, axis=1)

        accuracy = torch.mean((predictions == y_true).float())
        return accuracy


## Creating model

### BackPropagation

#### Model

In [7]:
X = torch.tensor([0.4, 0.9])
y = torch.tensor([0.15, 0.85])

### Neural Network Layers Initialization

In [8]:
hidden_layer_1 = DenseLayer(2, 2)
activation1 = Activation_ReLU()
output_layer = DenseLayer(2, 2)
activation2 = Activation_Sigmoid()

### Neural Network Forward Pass

In [9]:
hidden_layer_1.forward(X)
activation1.forward(hidden_layer_1.output)
output_layer.forward(activation1.output)
activation2.forward(output_layer.output)
activation2.output

tensor([[0.5000, 0.5000]])

### Backpropagation for a Two-Neuron Network

In [10]:
def back_prop_two_neuron(fp):
  """
  Perform backpropagation to update weights and biases for a two-neuron neural network.

  Parameters:
  - fp (torch.Tensor): Forward pass result, representing the output of the network.

  Returns:
  None
  """
   
  lr = torch.tensor(0.01)
  back1 = (fp[0][0]-y[0])*(1-fp[0][0])*fp[0][0]
  back2 = (fp[0][1]-y[1])*(1-fp[0][1])*fp[0][1]
  output_layer.weights[0][0] -= lr * back1*activation1.output[0][0]
  output_layer.weights[0][1] -= lr * back1*activation1.output[0][1]
  output_layer.weights[1][0] -= lr * back2*activation1.output[0][0]
  output_layer.weights[1][1] -= lr * back2*activation1.output[0][1]
  output_layer.biases[0][0] -= lr * back1
  output_layer.biases[0][1] -= lr * back2

  hidden_layer_1.weights[0][0] -= lr * (back1 * output_layer.weights[0][0] * X[0] + back2 * output_layer.weights[0][1] * X[0] ) if hidden_layer_1.output[0][0] > 0 else 0
  hidden_layer_1.weights[0][1] -= lr * (back1 * output_layer.weights[0][0] * X[1] + back2 * output_layer.weights[0][1] * X[1] ) if hidden_layer_1.output[0][0] > 0 else 0
  hidden_layer_1.weights[1][0] -= lr * (back1 * output_layer.weights[1][0] * X[0] + back2 * output_layer.weights[1][1] * X[0] ) if hidden_layer_1.output[0][1] > 0 else 0
  hidden_layer_1.weights[1][1] -= lr * (back1 * output_layer.weights[1][0] * X[1] + back2 * output_layer.weights[1][1] * X[1] ) if hidden_layer_1.output[0][1] > 0 else 0
  hidden_layer_1.biases[0][0] -= lr * (back1 * output_layer.weights[0][0] + back2 * output_layer.weights[0][1] ) if hidden_layer_1.output[0][0] > 0 else 0
  hidden_layer_1.biases[0][1] -= lr * (back1 * output_layer.weights[1][0] + back2 * output_layer.weights[1][1] ) if hidden_layer_1.output[0][1] > 0 else 0

In [12]:
print("Output after Backpropagation:")
back_prop_two_neuron(activation2.output)

print("\nHidden Layer 1:")
print("  - Weights:\n", hidden_layer_1.weights)
print("  - Biases:\n", hidden_layer_1.biases)

print("\nOutput Layer:")
print("  - Weights:\n", output_layer.weights)
print("  - Biases:\n", output_layer.biases)

Output after Backpropagation:

Hidden Layer 1:
  - Weights:
 tensor([[0.0052, 0.0065],
        [0.0018, 0.0048]])
  - Biases:
 tensor([[-6.9484e-06,  4.8669e-06]])

Output Layer:
  - Weights:
 tensor([[0.0084, 0.0044],
        [0.0072, 0.0100]])
  - Biases:
 tensor([[-0.0018,  0.0017]])
